In [156]:
import numpy as np
import pandas as pd
import os.path
import glob
import time


dataset_path = r"/Users/jaewon/Desktop/eicu/delirium-master/First 24 Hour/Dataset/"
eicu_path = r"/Users/jaewon/Desktop/eicu/"
feature_path = "/Users/jaewon/Desktop/eicu/delirium-master/First 24 Hour/Features/"
medication_path = feature_path+"Medications/"

comp_path = dataset_path
comp_filename_ver = "complete_patientstayid_label_list_0904.csv"

comp = pd.read_csv(dataset_path+comp_filename_ver)

In [2]:
comp

,patientunitstayid,Class,final_offset
0,242154,1,5640.0
1,242290,0,1362.0
2,242474,0,807.0
3,242505,1,5900.0
4,242714,0,2712.0
...,...,...,...
27058,3347353,1,4.0
27059,3347960,1,651.0
27060,3349086,1,574.0
27061,3349342,1,10390.0


### Anticholinergic_beforeAdmission

In [84]:
#define all the paths. 
drugPathList = glob.glob(str(medication_path + '/DrugNameLists/'+'*'))
treatmentPathList = glob.glob(str(medication_path+'/TreatmentStrings/'+'*'))

In [86]:
drugPath_AC = medication_path + '/DrugNameLists/Anticholinergics.csv'
treatmentPath_AC = medication_path + '/TreatmentStrings/Anticholinergics.csv'

In [113]:
# get infusion drug info
infu = pd.read_csv(eicu_path+"infusiondrug.csv", usecols=['patientunitstayid','infusionoffset','drugname'])
    
# Get medication table info
med = pd.read_csv(eicu_path+"medication.csv",usecols=['patientunitstayid', 'drugstartoffset', 'drugname', 'drughiclseqno', 'drugstopoffset','drugordercancelled'])
# remove cancelled orders
med = med[med['drugordercancelled']=='No']
# drop column with drug order info.
med.drop(columns=['drugordercancelled'],inplace=True)
 
# Get Treatment table info
treat = pd.read_csv(eicu_path+"treatment.csv",usecols=['patientunitstayid', 'treatmentoffset', 'treatmentstring'])
    
# only keep rows with the complete list of data ids
comp = pd.read_csv(dataset_path+comp_filename_ver)
compInfu = infu[infu['patientunitstayid'].isin(comp['patientunitstayid'])]
compMed = med[med['patientunitstayid'].isin(comp['patientunitstayid'])]
compTreat = treat[treat['patientunitstayid'].isin(comp['patientunitstayid'])]

In [114]:
# keep the rows from medication with relevant drugs. 
compMed = compMed.applymap(lambda s:s.lower() if type(s) == str else s)
drugMed = compMed[compMed['drugname'].str.contains('|'.join(druglist),na=False)]
hiclMed = compMed[compMed['drughiclseqno'].isin(hicllist)]
compMed = pd.concat([drugMed,hiclMed]).drop_duplicates()

In [115]:
print(compMed.shape)
print(drugMed.shape)
print(hiclMed.shape)
print(compMed.shape)

(3218, 5)
(3435, 5)
(0, 5)
(3218, 5)


In [116]:
# keep the rows from treatment with relevant drugs. 
# pull list of strings to search treatment with. 
treatStrings = pd.read_csv(treatmentPathList_AC)
treatStrings = treatStrings.applymap(lambda s:s.lower() if type(s) == str else s)
treatStringsList = treatStrings.values.astype(str).tolist()
treatStringsList = [item.lower() for sublist in treatStringsList for item in sublist]
    
compTreat = compTreat.applymap(lambda s:s.lower() if type(s) == str else s)
compTreat = compTreat[compTreat['treatmentstring'].str.contains('|'.join(treatStringsList),na=False)]
compTreat['drugstopoffset']=np.nan
compTreat['drughiclseqno']=np.nan
compTreat = compTreat.rename(columns={'treatmentoffset':'drugstartoffset','treatmentstring':'drugname'})

In [122]:
print(compTreat.shape)
compTreat.head()

(42, 5)


,patientunitstayid,drugstartoffset,drugname,drugstopoffset,drughiclseqno
1044472,960317,2177,neurologic|neuromyopathy therapy|anticholinerg...,NaN,NaN
1044491,960317,2177,neurologic|neuromyopathy therapy|anticholinerg...,NaN,NaN
1047110,961451,156,neurologic|neuromyopathy therapy|anticholinerg...,NaN,NaN
1055333,964144,1094,neurologic|neuromyopathy therapy|anticholinerg...,NaN,NaN
1055337,964144,1608,neurologic|neuromyopathy therapy|anticholinerg...,NaN,NaN


In [123]:
 # combine the treatment, medication, and infusion info together
compFeat = pd.concat([compMed,compInfu,compTreat],sort=False)
compFeat = compFeat.sort_values(by=['patientunitstayid','drugstartoffset','drugstopoffset'])
compFeat = compFeat[['patientunitstayid','drugstartoffset','drugstopoffset','drugname','drughiclseqno']]
compFeat

,patientunitstayid,drugstartoffset,drugstopoffset,drugname,drughiclseqno
14,242154,NaN,NaN,Propofol (mcg/kg/min),NaN
34,242505,NaN,NaN,Norepinephrine (mcg/min),NaN
35,242505,NaN,NaN,Norepinephrine (mcg/min),NaN
36,242505,NaN,NaN,Norepinephrine (mcg/min),NaN
37,242505,NaN,NaN,Norepinephrine (mcg/min),NaN
...,...,...,...,...,...
4798497,3349777,NaN,NaN,NiCARDipine (std) 50 mg Sodium Chloride 0.9% 5...,NaN
4798498,3349777,NaN,NaN,NiCARDipine (premix) 20 mg Sodium Chloride 0.9...,NaN
4798499,3349777,NaN,NaN,NiCARDipine (premix) 20 mg Sodium Chloride 0.9...,NaN
4798500,3349777,NaN,NaN,NiCARDipine (std) 50 mg Sodium Chloride 0.9% 5...,NaN


In [125]:
print("~24hr:",compFeat[compFeat['drugstartoffset'] <= 1440].shape[0])
print("~admission:",compFeat[compFeat['drugstartoffset'] <= 0].shape[0])

~24hr: 1844
~admission: 1083


In [127]:
# only keep them if the drug start offset was before admission. 
compFeat = compFeat[compFeat['drugstartoffset'] <= 0 ]

# get a list of the stay IDs that have the drug admissions.
compFeat = compFeat[['patientunitstayid']]
compFeat = compFeat.drop_duplicates()

compFeat

,patientunitstayid
5536165,280954
5592989,425148
1948547,425879
5594493,427245
5594740,427781
...,...
7114434,3106881
7116242,3108780
7116507,3108965
7130335,3124153


In [128]:
# Convert it to a true/false for each stay ID in comp
comp['Anticholinergics'] = comp['patientunitstayid'].isin(compFeat['patientunitstayid'])
comp

,patientunitstayid,Class,final_offset,Anticholinergics
0,242154,1,5640.0,False
1,242290,0,1362.0,False
2,242474,0,807.0,False
3,242505,1,5900.0,False
4,242714,0,2712.0,False
...,...,...,...,...
27058,3347353,1,4.0,False
27059,3347960,1,651.0,False
27060,3349086,1,574.0,False
27061,3349342,1,10390.0,False


In [131]:
Antich = comp[['patientunitstayid','Anticholinergics']]
print("# Anticholinergic treatment that has started before the admission:", Antich.Anticholinergics.sum())
Antich

# Anticholinergic treatment that has started before the admission: 874


,patientunitstayid,Anticholinergics
0,242154,False
1,242290,False
2,242474,False
3,242505,False
4,242714,False
...,...,...
27058,3347353,False
27059,3347960,False
27060,3349086,False
27061,3349342,False


In [132]:
comp.to_csv(medication_path+"Anticholinergics_beforeAdmission.csv")

### 24hrAllDrugFeatures

In [133]:
#define all the paths. 
drugPathList = glob.glob(str(medication_path + '/DrugNameLists/'+'*'))
treatmentPathList = glob.glob(str(medication_path+'/TreatmentStrings/'+'*'))

In [154]:
def FirstdayDrugFeature(drugSearchListPath,treatmentSearchListPath):
    
    ## importing drug/medication/treatment info files & paths

    # get infusion drug info
    infu = pd.read_csv(eicu_path+"infusiondrug.csv", usecols=['patientunitstayid','infusionoffset','drugname'])
    # Get Treatment table info
    treat = pd.read_csv(eicu_path+"treatment.csv",usecols=['patientunitstayid', 'treatmentoffset', 'treatmentstring'])

    # Get medication table info
    med = pd.read_csv(eicu_path+"medication.csv",usecols=['patientunitstayid', 'drugstartoffset', 'drugname', 'drughiclseqno', 'drugstopoffset','drugordercancelled'])
    med = med[med['drugordercancelled']=='No'] # remove cancelled orders
    med.drop(columns=['drugordercancelled'],inplace=True) # drop column with drug order info
 
    # only keep rows with the complete list of data ids
    comp = pd.read_csv(dataset_path+comp_filename_ver)
    compInfu = infu[infu['patientunitstayid'].isin(comp['patientunitstayid'])]
    compMed = med[med['patientunitstayid'].isin(comp['patientunitstayid'])]
    compTreat = treat[treat['patientunitstayid'].isin(comp['patientunitstayid'])]
    
    ## only keep rows with relevant drugs
    # import lists of drug names to search for, and make it all lowercase
    # change this part to get different drug features
    drug=pd.read_csv(drugSearchListPath)
    drug = drug.applymap(lambda s:s.lower() if type(s) == str else s)
    druglist = drug.values.astype(str).tolist()
    druglist = [item.lower() for sublist in druglist for item in sublist]

    # this csv was generated using Create HICL Drug Name Legend.py
    hicl = pd.read_csv(medication_path + "HICLlegend.csv")
    # make it all lowercase
    hicl = hicl.applymap(lambda s:s.lower() if type(s) == str else s)
    # pull relevant HICL codes
    hicl = hicl[hicl['drugname'].str.contains('|'.join(druglist))]
    hicl = hicl.drop(columns=['drugname'])
    hicl = hicl.drop_duplicates()
    hicllist = hicl.values.astype(float).tolist()

    # keep the rows from medication with relevant drugs.
    compMed = compMed.applymap(lambda s:s.lower() if type(s) == str else s)
    drugMed = compMed[compMed['drugname'].str.contains('|'.join(druglist),na=False)]
    hiclMed = compMed[compMed['drughiclseqno'].isin(hicllist)]
    compMed = pd.concat([drugMed,hiclMed])
    compMed = compMed.drop_duplicates()

    # keep the rows from infusion with relevant drugs.
    compInfu = compInfu.applymap(lambda s:s.lower() if type(s) == str else s)
    compInfu = compInfu[compInfu['drugname'].str.contains('|'.join(druglist),na=False)]
    compInfu['drugstopoffset']=np.nan
    compInfu['drughiclseqno']=np.nan
    compInfu = compInfu.rename(columns={'infusionoffset':'drugstartoffset'})

    # keep the rows from treatment with relevant drugs.
    # pull list of strings to search treatment with.
    treatStrings = pd.read_csv(treatmentSearchListPath)
    treatStrings = treatStrings.applymap(lambda s:s.lower() if type(s) == str else s)
    treatStringsList = treatStrings.values.astype(str).tolist()
    treatStringsList = [item.lower() for sublist in treatStringsList for item in sublist]

    compTreat = compTreat.applymap(lambda s:s.lower() if type(s) == str else s)
    compTreat = compTreat[compTreat['treatmentstring'].str.contains('|'.join(treatStringsList),na=False)]
    compTreat['drugstopoffset']=np.nan
    compTreat['drughiclseqno']=np.nan
    compTreat = compTreat.rename(columns={'treatmentoffset':'drugstartoffset','treatmentstring':'drugname'})

    # combine the treatment, medication, and infusion info together
    compFeat = pd.concat([compMed,compInfu,compTreat],sort=False)
    compFeat = compFeat.sort_values(by=['patientunitstayid','drugstartoffset','drugstopoffset'])
    compFeat = compFeat[['patientunitstayid','drugstartoffset','drugstopoffset','drugname','drughiclseqno']]

    # only keep them if the drug start offset was before 24hr of stay.
    compFeat = compFeat[compFeat['drugstartoffset'] <= 1440 ]
    # compFeat = compFeat[compFeat['drugstartoffset'] >= 0 ]

    # get a list of the stay IDs that have the drug admissions.
    compFeat = compFeat[['patientunitstayid']]
    compFeat = compFeat.drop_duplicates()

    # Convert it to a true/false for each stay ID in comp
    newColName = '24hrMed_' + drug.columns.values[0]
    comp[newColName] = comp['patientunitstayid'].isin(compFeat['patientunitstayid'])
    
    return comp[newColName]

- deleted the >=0 boundary

In [157]:
comp

,patientunitstayid,Class,final_offset
0,242154,1,5640.0
1,242290,0,1362.0
2,242474,0,807.0
3,242505,1,5900.0
4,242714,0,2712.0
...,...,...,...
27058,3347353,1,4.0
27059,3347960,1,651.0
27060,3349086,1,574.0
27061,3349342,1,10390.0


In [158]:
#error checking
if len(drugPathList)!=len(treatmentPathList):
    raise NameError('you goofin fam, the lists of paths are different lengths')

for path in drugPathList:
    if os.path.isfile(path) == False:
        raise NameError(path+" is not a valid file path")
        
for path in treatmentPathList:
    if os.path.isfile(path) == False:
        raise NameError(path+" is not a valid file path")

In [159]:
start_time = time.time()
comp = pd.DataFrame(comp['patientunitstayid'])

for i in range(1,len(drugPathList)):
    comp = pd.concat(objs=[comp,FirstdayDrugFeature(
        drugPathList[i],treatmentPathList[i])],axis = 1)

end_time = time.time()
total_time = end_time - start_time
print("Total time taken: {:.2f} mins".format(total_time / 60))
comp

/Users/jaewon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/jaewon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/jaewon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


Total time taken: 14.47 mins


,patientunitstayid,24hrMed_Vasopressors,24hrMed_Cephalosporins,24hrMed_SNRIAntidepressants,24hrMed_Quinolones,24hrMed_Antihistamines,24hrMed_aminoglycosides,24hrMed_CalciumChannelBlockers,24hrMed_Lincomycins,24hrMed_AnticholinergicBronchodilators,...,24hrMed_ColloidFluids,24hrMed_SSRIAntidepressants,24hrMed_MiscAntidepressants,24hrMed_Methylxanthines,24hrMed_NSAIDs,24hrMed_PhenylpiperazineAntidepressants,24hrMed_Somatostatin,24hrMed_ProtonPumpInhibitor,24hrMed_AdrenergicBronchodilators,24hrMed_GeneralAnesthetics
0,242154,False,False,False,False,False,False,False,False,True,...,False,True,True,False,False,True,True,False,True,True
1,242290,False,True,False,False,False,False,False,False,True,...,False,True,False,False,False,True,True,True,True,True
2,242474,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,True,False,False
3,242505,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,True,True,False
4,242714,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,3347353,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
27059,3347960,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,True,False,False
27060,3349086,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,True,True,True,False
27061,3349342,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,True,True,False,False


In [160]:
comp.columns

Index(['patientunitstayid', '24hrMed_Vasopressors', '24hrMed_Cephalosporins',
       '24hrMed_SNRIAntidepressants', '24hrMed_Quinolones',
       '24hrMed_Antihistamines', '24hrMed_aminoglycosides',
       '24hrMed_CalciumChannelBlockers', '24hrMed_Lincomycins',
       '24hrMed_AnticholinergicBronchodilators', '24hrMed_Carbapenems',
       '24hrMed_Anticoagulants', '24hrMed_Sulfonamides', '24hrMed_Diuretics',
       '24hrMed_Macrolides', '24hrMed_Antidiarrheals', '24hrMed_Barbiturates',
       '24hrMed_Vasodilators', '24hrMed_GlucoseElevating', '24hrMed_Opioids',
       '24hrMed_Insulins', '24hrMed_Penicillins', '24hrMed_Haloperidol',
       '24hrMed_TetracyclicAntidepressants', '24hrMed_Tetracyclines',
       '24hrMed_CrystalloidFluids', '24hrMed_TricyclicAntidepressants',
       '24hrMed_MAOIAntidepressants', '24hrMed_Glucocorticoids',
       '24hrMed_Thrombolytics', '24hrMed_Benzodiazepines',
       '24hrMed_ClassVAntiarrhythmics', '24hrMed_BetaBlockers',
       '24hrMed_Anticholiner

In [161]:
# Save off the complete dataframe of 24hrs medication info.
comp.to_csv('24hrsAllDrugFeatures.csv')

### Exploration

In [39]:
def DrugFeature_offset(drugSearchListPath,treatmentSearchListPath):

    # get infusion drug info
    infu = pd.read_csv(eicu_path+"infusionDrug.csv", usecols=['patientunitstayid','infusionoffset','drugname'])

    # Get medication table info
    med = pd.read_csv(eicu_path+"medication.csv",usecols=['patientunitstayid', 'drugstartoffset', 'drugname', 'drughiclseqno', 'drugstopoffset','drugordercancelled'])
    # remove cancelled orders
    med = med[med['drugordercancelled']=='No']
    # drop column with drug order info.
    med.drop(columns=['drugordercancelled'],inplace=True)

    # Get Treatment table info
    treat = pd.read_csv(eicu_path+"treatment.csv",usecols=['patientunitstayid', 'treatmentoffset', 'treatmentstring'])

    # only keep rows with the complete list of data ids
    comp = pd.read_csv(comp_path+comp_filename_ver)
    compInfu = infu[infu['patientunitstayid'].isin(comp['patientunitstayid'])]
    compMed = med[med['patientunitstayid'].isin(comp['patientunitstayid'])]
    compTreat = treat[treat['patientunitstayid'].isin(comp['patientunitstayid'])]

    # only keep rows with relevant drugs

    # import lists of drug names to search for, and make it all lowercase
    # change this part to get different drug features
    drug=pd.read_csv(drugSearchListPath)
    drug = drug.applymap(lambda s:s.lower() if type(s) == str else s)
    druglist = drug.values.astype(str).tolist()
    druglist = [item.lower() for sublist in druglist for item in sublist]

    # this csv was generated using Create HICL Drug Name Legend.py
    hicl = pd.read_csv(medication_path + "HICLlegend.csv")
    # make it all lowercase
    hicl = hicl.applymap(lambda s:s.lower() if type(s) == str else s)
    # pull relevant HICL codes
    hicl = hicl[hicl['drugname'].str.contains('|'.join(druglist))]
    hicl = hicl.drop(columns=['drugname'])
    hicl = hicl.drop_duplicates()
    hicllist = hicl.values.astype(float).tolist()

    # keep the rows from medication with relevant drugs.
    compMed = compMed.applymap(lambda s:s.lower() if type(s) == str else s)
    drugMed = compMed[compMed['drugname'].str.contains('|'.join(druglist),na=False)]
    hiclMed = compMed[compMed['drughiclseqno'].isin(hicllist)]
    compMed = pd.concat([drugMed,hiclMed])
    compMed = compMed.drop_duplicates()

    # keep the rows from infusion with relevant drugs.
    compInfu = compInfu.applymap(lambda s:s.lower() if type(s) == str else s)
    compInfu = compInfu[compInfu['drugname'].str.contains('|'.join(druglist),na=False)]
    compInfu['drugstopoffset']=np.nan
    compInfu['drughiclseqno']=np.nan
    compInfu = compInfu.rename(columns={'infusionoffset':'drugstartoffset'})

    # keep the rows from treatment with relevant drugs.
    # pull list of strings to search treatment with.
    treatStrings = pd.read_csv(treatmentSearchListPath)
    treatStrings = treatStrings.applymap(lambda s:s.lower() if type(s) == str else s)
    treatStringsList = treatStrings.values.astype(str).tolist()
    treatStringsList = [item.lower() for sublist in treatStringsList for item in sublist]

    compTreat = compTreat.applymap(lambda s:s.lower() if type(s) == str else s)
    compTreat = compTreat[compTreat['treatmentstring'].str.contains('|'.join(treatStringsList),na=False)]
    compTreat['drugstopoffset']=np.nan
    compTreat['drughiclseqno']=np.nan
    compTreat = compTreat.rename(columns={'treatmentoffset':'drugstartoffset','treatmentstring':'drugname'})

    # combine the treatment, medication, and infusion info together
    compFeat = pd.concat([compMed,compInfu,compTreat],sort=False)
    compFeat = compFeat.sort_values(by=['patientunitstayid','drugstartoffset','drugstopoffset'])
    compFeat = compFeat[['patientunitstayid','drugstartoffset','drugstopoffset','drugname','drughiclseqno']]

    # only keep them if the drug start offset was before 24hr of stay.
    compFeat = compFeat[compFeat['drugstartoffset'] <= 1440 ]

    # get a list of the stay IDs that have the drug admissions.
    compFeat_id = compFeat[['patientunitstayid']]
    compFeat_id = compFeat.drop_duplicates()
    
    # Get the minimum drug start offset for each patientunitstayid
    min_drugstartoffset = compFeat.groupby('patientunitstayid')['drugstartoffset'].min()
    # Convert it to a DataFrame and reset index
    min_drugstartoffset_df = min_drugstartoffset.reset_index()
    # Reset index of min_drugstartoffset_df
    min_drugstartoffset_df.reset_index(drop=True, inplace=True)



    # Convert it to a true/false for each stay ID in comp
    newColName = 'FirstDay' + drug.columns.values[0]
    comp[newColName] = comp['patientunitstayid'].isin(compFeat_id['patientunitstayid'])

    # Set values of compFeat['drugstartoffset'] as comp[newColName] values
    comp.loc[comp['patientunitstayid'].isin(min_drugstartoffset_df['patientunitstayid']), newColName] = min_drugstartoffset_df.loc[comp['patientunitstayid'].isin(min_drugstartoffset_df['patientunitstayid']), 'drugstartoffset']

    return comp[newColName]

In [40]:
#error checking
if len(drugPathList)!=len(treatmentPathList):
    raise NameError('you goofin fam, the lists of paths are different lengths')

for path in drugPathList:
    if os.path.isfile(path) == False:
        raise NameError(path+" is not a valid file path")

for path in treatmentPathList:
    if os.path.isfile(path) == False:
        raise NameError(path+" is not a valid file path")

In [ ]:
comp = pd.DataFrame(comp['patientunitstayid'])
for i in range(1,len(drugPathList)):
    comp = pd.concat(objs=[comp,DrugFeature_offset(
        drugPathList[i],treatmentPathList[i])],axis = 1)

In [80]:
# print out the # cases before admission, # cases admission~24hr
for col in comp.columns:
    print(col,":",comp[comp[col].astype(float)<0].shape[0],",",comp[(comp[col].astype(float)<=1440)&(comp[col].astype(float)>=0)].shape[0])

patientunitstayid : 0 , 0
FirstDayVasopressors : 790 , 22481
FirstDayCephalosporins : 398 , 23739
FirstDaySNRIAntidepressants : 0 , 26957
FirstDayQuinolones : 28 , 25931
FirstDayAntihistamines : 204 , 24136
FirstDayaminoglycosides : 0 , 26992
FirstDayCalciumChannelBlockers : 114 , 24302
FirstDayLincomycins : 0 , 26847
FirstDayAnticholinergicBronchodilators : 691 , 23152
FirstDayCarbapenems : 3 , 26169
FirstDayAnticoagulants : 2817 , 18141
FirstDaySulfonamides : 0 , 27061
FirstDayDiuretics : 744 , 22091
FirstDayMacrolides : 170 , 21406
FirstDayAntidiarrheals : 1 , 26749
FirstDayBarbiturates : 0 , 26417
FirstDayVasodilators : 1407 , 20605
FirstDayGlucoseElevating : 3820 , 21484
FirstDayOpioids : 4615 , 15878
FirstDayInsulins : 1252 , 19770
FirstDayPenicillins : 1 , 25847
FirstDayHaloperidol : 20 , 26024
FirstDayTetracyclicAntidepressants : 4 , 24260
FirstDayTetracyclines : 0 , 27003
FirstDayCrystalloidFluids : 10912 , 12555
FirstDayTricyclicAntidepressants : 3 , 25936
FirstDayMAOIAntidep